In [1]:
############################################################################################################################
#############################                     convert data into df                    ##################################
############################################################################################################################


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrices
import statsmodels.api as sm


from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# read data as framework
df = pd.read_csv('./data_integer_shift_column.csv')

# check the framework

#df.head()

In [2]:
############################## according to step 2 the following attributes are important for regression

                                #'tvd (ft)',
                                # 'well spacing',
                                # 'log permeability',
                                # 'porosity',
                                # 'poisson ratio',
                                # 'p-velocity',
                                # 's-velocity', 
                                # 'youngs modulus', 
                                # 'total number of stages',
                                # 'proppant volume',
                                # 'date_sec',
                                # 'op_number',
                                # 'production'

############################## according to step 3 adj R2 was too small ~0.55


############################## thus, the model need to be improved


In [3]:
############################################################################################################################
###################          use fix effect to improve the model and increase adj R2           ##########################################
######################################################################################################################



##################### record the attributes and use the orginal data in the attributes to repeat the adj R 2

df_att = df[['tvd (ft)',
             'well spacing',
             'log permeability',
             'porosity',
             'poisson ratio',
             'p-velocity',
             's-velocity', 
             'youngs modulus', 
             'total number of stages',
             'proppant volume',
             'date_sec',
              'op_number',
             'production']]
df_att_drop = df_att.dropna()


##################### set x and y for the models
x = df_att_drop.drop(['production'], axis=1)
y = df_att_drop['production']

In [4]:
###############################  use critical varables to predict, find adj R2 BEFORE 'fixed effect' function
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.552
Method:                 Least Squares   F-statistic:                     64.75
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          2.60e-100
Time:                        16:33:25   Log-Likelihood:                -5142.6
No. Observations:                 621   AIC:                         1.031e+04
Df Residuals:                     608   BIC:                         1.037e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   7975

/Users/yanxu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [5]:
#### The adj R2 before improvement was 0.552

In [6]:
###############################  use critical varables to predict, find adj R2 before 'fixed effect' function

df_FE = sm.OLS(endog=y, exog=x, entity_effects=True).fit()
print(df_FE.summary())

                                 OLS Regression Results                                
Dep. Variable:             production   R-squared (uncentered):                   0.848
Model:                            OLS   Adj. R-squared (uncentered):              0.845
Method:                 Least Squares   F-statistic:                              282.1
Date:                Tue, 27 Oct 2020   Prob (F-statistic):                   1.97e-239
Time:                        16:33:25   Log-Likelihood:                         -5149.6
No. Observations:                 621   AIC:                                  1.032e+04
Df Residuals:                     609   BIC:                                  1.038e+04
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

In [7]:
# Another syntex for Fix effects


# FE_ols = sm.OLS(endog=y, exog=x, formula='logrealwage ~ HHI+C(Cz)+C(yQ)-1',
#                               data=df_att_drop).fit()
# print(FE_ols.summary())

In [8]:
#########################################     find the attribute with too large P value (P >0.1)

### according to reference, if |P value| < 0.05, the feature is significant
### to be conservative, we choose |P value| < 0.1, and delete the value =>0.1


### so we need to delete 'operator'

In [9]:
#########################################     delete 'operator'

df_att_drop_2 = df_att_drop.drop(['op_number'], axis=1)

In [10]:
##################### set x and y for the models
x = df_att_drop_2.drop(['production'], axis=1)
y = df_att_drop_2['production']

In [11]:
###############################  use critical varables to predict, find adj R2 before 'fixed effect' function

df_FE = sm.OLS(endog=y, exog=x, entity_effects=True).fit()
print(df_FE.summary())

                                 OLS Regression Results                                
Dep. Variable:             production   R-squared (uncentered):                   0.847
Model:                            OLS   Adj. R-squared (uncentered):              0.844
Method:                 Least Squares   F-statistic:                              307.0
Date:                Tue, 27 Oct 2020   Prob (F-statistic):                   3.13e-240
Time:                        16:33:25   Log-Likelihood:                         -5150.7
No. Observations:                 621   AIC:                                  1.032e+04
Df Residuals:                     610   BIC:                                  1.037e+04
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

In [12]:
#### after deleting 'op_number', i.e., 'operator', the changes in R2 was negligible
#### this indicates that our deletion was reasonable -  does not hurt the model

In [13]:
############################################################################################################################
###############################                     find coefficient            ############################################
############################################################################################################################


###############################          covert summary table into DataFrame  
rslt_sum = df_FE.summary().tables[1]
df_rslt_sum = pd.DataFrame(rslt_sum)
df_rslt_sum

,0,1,2,3,4,5,6
0,,coef,std err,t,P>|t|,[0.025,0.975]
1,tvd (ft),0.4592,0.052,8.759,0.000,0.356,0.562
2,well spacing,0.0758,0.031,2.428,0.015,0.014,0.137
3,log permeability,-300.4283,145.736,-2.061,0.040,-586.633,-14.223
4,porosity,-4302.5718,2286.406,-1.882,0.060,-8792.755,187.611
5,poisson ratio,8105.0813,2176.692,3.724,0.000,3830.362,1.24e+04
6,p-velocity,-0.4963,0.085,-5.841,0.000,-0.663,-0.329
7,s-velocity,2.0642,0.334,6.175,0.000,1.408,2.721
8,youngs modulus,-566.1183,71.376,-7.932,0.000,-706.290,-425.947
9,total number of stages,17.3564,4.400,3.945,0.000,8.716,25.997


In [14]:
###############################          use the first raw as the columns name  

#grab the first row for the header
new_header = df_rslt_sum.iloc[0] 

#take the data less the header row
df_rslt_sum = df_rslt_sum[1:] 

#set the header row as the df header
df_rslt_sum.columns = new_header

df_rslt_sum

,,coef,std err,t,P>|t|,[0.025,0.975]
1,tvd (ft),0.4592,0.052,8.759,0.000,0.356,0.562
2,well spacing,0.0758,0.031,2.428,0.015,0.014,0.137
3,log permeability,-300.4283,145.736,-2.061,0.040,-586.633,-14.223
4,porosity,-4302.5718,2286.406,-1.882,0.060,-8792.755,187.611
5,poisson ratio,8105.0813,2176.692,3.724,0.000,3830.362,1.24e+04
6,p-velocity,-0.4963,0.085,-5.841,0.000,-0.663,-0.329
7,s-velocity,2.0642,0.334,6.175,0.000,1.408,2.721
8,youngs modulus,-566.1183,71.376,-7.932,0.000,-706.290,-425.947
9,total number of stages,17.3564,4.400,3.945,0.000,8.716,25.997
10,proppant volume,8.879e-05,1.07e-05,8.310,0.000,6.78e-05,0.000


In [15]:
###############################          use the first column as the index

#rename the first column
df_rslt_sum_new = df_rslt_sum.rename(columns={ df_rslt_sum.columns[0]: 'features' })

df_rslt_sum_new

,features,coef,std err,t,P>|t|,[0.025,0.975]
1,tvd (ft),0.4592,0.052,8.759,0.000,0.356,0.562
2,well spacing,0.0758,0.031,2.428,0.015,0.014,0.137
3,log permeability,-300.4283,145.736,-2.061,0.040,-586.633,-14.223
4,porosity,-4302.5718,2286.406,-1.882,0.060,-8792.755,187.611
5,poisson ratio,8105.0813,2176.692,3.724,0.000,3830.362,1.24e+04
6,p-velocity,-0.4963,0.085,-5.841,0.000,-0.663,-0.329
7,s-velocity,2.0642,0.334,6.175,0.000,1.408,2.721
8,youngs modulus,-566.1183,71.376,-7.932,0.000,-706.290,-425.947
9,total number of stages,17.3564,4.400,3.945,0.000,8.716,25.997
10,proppant volume,8.879e-05,1.07e-05,8.310,0.000,6.78e-05,0.000


In [17]:
# list the data type
list(df_rslt_sum_new.columns) 

['features',

In [18]:
# convert DataFrame as CSV file and then convert back to change the 'statsmodels.iolib.table.Cell' format

df_rslt_sum_new.to_csv('df_transition.csv',index=False)

# read data as framework
df_transition = pd.read_csv('./df_transition.csv')

In [19]:
############################################################################################################################
###############################                     result            ############################################
############################################################################################################################

## R2 was 0.847
## adj R2 was 0.844

# read data as framework
df_transition[['features','coef','P>|t|']]

,features,coef,P>|t|
0,tvd (ft),0.459200,0.000
1,well spacing,0.075800,0.015
2,log permeability,-300.428300,0.040
3,porosity,-4302.571800,0.060
4,poisson ratio,8105.081300,0.000
5,p-velocity,-0.496300,0.000
6,s-velocity,2.064200,0.000
7,youngs modulus,-566.118300,0.000
8,total number of stages,17.356400,0.000
9,proppant volume,0.000089,0.000
